# Automated ML

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

In [1]:
from azureml.core import Workspace, Experiment
from azureml.data.dataset_factory import TabularDatasetFactory
from azureml.train.automl import AutoMLConfig
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException


## Dataset

### Overview
TODO: In this markdown cell, give an overview of the dataset you are using. Also mention the task you will be performing.


TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [2]:
ws = Workspace.from_config()

# choose a name for experiment
experiment_name = 'automl'

experiment=Experiment(ws, experiment_name)

Performing interactive authentication. Please follow the instructions on the terminal.
To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code RZF9EYE3E to authenticate.
You have logged in. Now let us find all the subscriptions to which you have access...
Interactive authentication successfully completed.


In [3]:
ds = TabularDatasetFactory.from_delimited_files(path = "https://www.kaggle.com/andrewmvd/heart-failure-clinical-data?select=heart_failure_clinical_records_dataset.csv", separator = ',', encoding = 'utf8')

In [15]:
import pandas as pd
ds.to_pandas_dataframe()

,<!DOCTYPE html>,Column2,Column3,Column4,Column5,Column6,Column7,Column8,Column9,Column10,...,Column1177,Column1178,Column1179,Column1180,Column1181,Column1182,Column1183,Column1184,Column1185,Column1186
0,"<html lang=""en"">",None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
1,<head>,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
2,<title>Heart Failure Prediction | Kaggle</...,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
3,"<meta charset=""utf-8"" />",None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
4,"<meta name=""robots"" content=""index","follow"" />",None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
243,None,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
244,</main>,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None
245,</body>,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,None


## AutoML Configuration

TODO: Explain why you chose the automl settings and cofiguration you used below.

In [5]:
cpu_cluster_name = "computecluster"

# Verify if the clusters already exists
try:
    cpu_cluster = ComputeTarget(workspace=ws, name=cpu_cluster_name)
    print('Found existing target, use it')
except ComputeTargetException:
    compute_config = AmlCompute.provisioning_configuration(vm_size = 'STANDARD_D2_V2', max_nodes = 4)
    cpu_cluster = ComputeTarget.create(ws, cpu_cluster_name, compute_config)

cpu_cluster.wait_for_completion(show_output = True)

Creating
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned


In [10]:
# TODO: Put your automl settings here
automl_settings = {
    "experiment_timeout_minutes":30,
    "task":'classification',
    "primary_metric":'accuracy',
    "training_data":ds,
    "label_column_name":'DEATH_EVENT',
    "n_cross_validations":5,
    "max_concurrent_iterations": 4,
    "featurization": 'auto'}

# TODO: Put your automl config here
automl_config = AutoMLConfig(
    compute_target=cpu_cluster,
    task = 'classification')

In [13]:
# TODO: Submit your experiment
remote_run = experiment.submit(automl_config, show_output = True)

ConfigException: ConfigException:
	Message: Input of type 'Unknown' is not supported. Supported types: [azureml.data.tabular_dataset.TabularDataset, azureml.pipeline.core.pipeline_output_dataset.PipelineOutputTabularDataset]
	InnerException: None
	ErrorResponse 
{
    "error": {
        "code": "UserError",
        "message": "Input of type 'Unknown' is not supported. Supported types: [azureml.data.tabular_dataset.TabularDataset, azureml.pipeline.core.pipeline_output_dataset.PipelineOutputTabularDataset]",
        "details_uri": "https://aka.ms/AutoMLConfig",
        "target": "training_data",
        "inner_error": {
            "code": "BadArgument",
            "inner_error": {
                "code": "ArgumentInvalid",
                "inner_error": {
                    "code": "InvalidInputDatatype"
                }
            }
        }
    }
}

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

In [ ]:
from azureml.widgets import RunDetails
RunDetails(remote_run).show()

## Best Model

TODO: In the cell below, get the best model from the automl experiments and display all the properties of the model.



In [ ]:
# Retrieve and save your best automl model.
best_run, best_model = remote_run.get_output()
print(best_run)

In [ ]:
print(best_model)


In [ ]:
#TODO: Save the best model
automl_model = best_run.register_model(model_name = 'best_automl_model.pkl', model_path = './outputs/')

In [ ]:
cpu_cluster.delete()

## Model Deployment

Remember you have to deploy only one of the two models you trained.. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service